# **Introduction to SKALE Checks**

The `skale-checks` library is a Python package designed for performing checks on SKALE nodes. It includes functionalities to verify node health, perform contract checks, and run watchdog checks to monitor node compatibility and ensure compliance.

https://github.com/skalenetwork/skale-checks

## **Table of Contents**

1. Installation
2. Architecture
3. Setting Up Environment
5. Watchdog Checks
4. Node Checks
5. Watchdog Client

## **Architecture**

```
NodeChecks (inherits WatchdogChecks)
│
├── WatchdogChecks (inherits BaseChecks)
│   ├── BaseChecks
│   └── Watchdog (is used in WatchdogChecks)
```

## **Installation**

First, install the `skale-checks` library using `pip`. Ensure you have Python 3.7 or later installed.

In [10]:
!pip install skale-checks

You should consider upgrading via the '/Users/rikon/dev/skale-checks/venv/bin/python -m pip install --upgrade pip' command.


## **Setting Up Environment**
To interact with the SKALE network, you'll need to set up a connection using the skale library. Replace the placeholders with your actual SKALE credentials and endpoints.

In [75]:
!curl -o /tmp/abi.json https://raw.githubusercontent.com/skalenetwork/skale-network/master/releases/mainnet/skale-manager/1.11.0/skale-manager-1.11.0-mainnet-abi.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  463k  100  463k    0     0  1866k      0 --:--:-- --:--:-- --:--:-- 1869k


In [76]:
from skale import Skale

ENDPOINT = 'https://ethereum-rpc.publicnode.com'

skale = Skale(ENDPOINT, abi_filepath='/tmp/abi.json')

web3 = Web3(Web3.HTTPProvider(ENDPOINT))

## **WatchdogChecks**

WatchdogChecks object represents and handles checks that could be retrieved via watchdog API. Let's choose one of the nodes from Mainnet and play with it

In [66]:
from skale_checks.checks.watchdog import WatchdogChecks

node_ip = '35.205.78.215'
domain_name = 'block-node-4.skale.bdnodes.net'

wd_checks = WatchdogChecks(
    node_ip,
    network='mainnet',  # or 'testnet'
    domain_name=domain_name,  # Optional
    web3=web3,  # Optional Web3 instance for endpoint checks
    timeout=10  # Optional timeout for connections
)

### Info()
WatchdogChecks obj has method info() to get the list of available checks

In [44]:
wd_checks.info()

{'btrfs': ['btrfs'],
 'core': ['core'],
 'endpoint': ['endpoint', 'trusted_endpoint', 'endpoint_speed'],
 'hardware': ['hardware'],
 'public_ip': ['public_ip'],
 'sgx': ['sgx', 'sgx_version'],
 'ssl': ['ssl'],
 'validator_nodes': ['validator_nodes'],
 'versions': ['versions']}

### Get()

All interactions with WatchdogChecks are performed via method get(). The returned object is a ChecksDict - a dict, where key is a check name and value is CheckStatus. 

CheckStatus could have 3 states:
- FAILED (0)
- PASSED (1)
- UNKNOWN (2) 

In [71]:
from skale_checks.checks.types import CheckStatus

In [47]:
wd_checks.get('endpoint')

{'endpoint': <CheckStatus.PASSED: 1>,
 'trusted_endpoint': <CheckStatus.PASSED: 1>,
 'endpoint_speed': <CheckStatus.PASSED: 1>}

In [43]:
wd_checks.get(exclude='versions')

{'core': <CheckStatus.PASSED: 1>,
 'btrfs': <CheckStatus.PASSED: 1>,
 'endpoint': <CheckStatus.PASSED: 1>,
 'trusted_endpoint': <CheckStatus.PASSED: 1>,
 'endpoint_speed': <CheckStatus.PASSED: 1>,
 'hardware': <CheckStatus.PASSED: 1>,
 'public_ip': <CheckStatus.PASSED: 1>,
 'sgx': <CheckStatus.PASSED: 1>,
 'sgx_version': <CheckStatus.FAILED: 0>,
 'ssl': <CheckStatus.PASSED: 1>,
 'validator_nodes': <CheckStatus.PASSED: 1>}

In [48]:
test_checks = wd_checks.get('versions','core','sgx')

In [70]:
if test_checks['core'] == CheckStatus.PASSED:
    print('OK')
else:
    print('NOT OK')

OK


## **NodeChecks**

NodeChecks is a higher-level abstraction above WatchdogChecks and represents all the checks for a node. It includes checks from Watchdog, Skale Manager, and Elasticsearch. An object of skale is required to initiate interaction.

In [67]:
from skale_checks.checks.node import NodeChecks

node_id = 220

node_checks = NodeChecks(
    skale,
    node_id,
    network='mainnet',  # or 'testnet'
    timeout=10  # Optional timeout for connections
)

In [57]:
node_checks.info()

{'btrfs': ['btrfs'],
 'core': ['core'],
 'endpoint': ['endpoint', 'trusted_endpoint', 'endpoint_speed'],
 'hardware': ['hardware'],
 'internal_ports': ['internal_ports'],
 'logs': ['logs'],
 'node_balance': ['node_balance'],
 'public_ip': ['public_ip'],
 'sgx': ['sgx', 'sgx_version'],
 'ssl': ['ssl'],
 'status': ['status'],
 'validator_balance': ['val_balance'],
 'validator_nodes': ['validator_nodes'],
 'versions': ['versions']}

In [58]:
node_checks.get()

{'btrfs': <CheckStatus.PASSED: 1>,
 'core': <CheckStatus.PASSED: 1>,
 'endpoint': <CheckStatus.PASSED: 1>,
 'trusted_endpoint': <CheckStatus.PASSED: 1>,
 'endpoint_speed': <CheckStatus.PASSED: 1>,
 'logs': <CheckStatus.UNKNOWN: 2>,
 'hardware': <CheckStatus.PASSED: 1>,
 'public_ip': <CheckStatus.PASSED: 1>,
 'node_balance': <CheckStatus.PASSED: 1>,
 'sgx': <CheckStatus.PASSED: 1>,
 'sgx_version': <CheckStatus.FAILED: 0>,
 'status': <CheckStatus.FAILED: 0>,
 'ssl': <CheckStatus.PASSED: 1>,
 'validator_nodes': <CheckStatus.PASSED: 1>,
 'versions': <CheckStatus.FAILED: 0>,
 'val_balance': <CheckStatus.PASSED: 1>,
 'internal_ports': <CheckStatus.PASSED: 1>}

In [60]:
node_checks.get(exclude='validator_nodes')

{'core': <CheckStatus.PASSED: 1>,
 'hardware': <CheckStatus.PASSED: 1>,
 'btrfs': <CheckStatus.PASSED: 1>,
 'logs': <CheckStatus.UNKNOWN: 2>,
 'endpoint': <CheckStatus.PASSED: 1>,
 'trusted_endpoint': <CheckStatus.PASSED: 1>,
 'endpoint_speed': <CheckStatus.PASSED: 1>,
 'node_balance': <CheckStatus.PASSED: 1>,
 'public_ip': <CheckStatus.PASSED: 1>,
 'sgx': <CheckStatus.PASSED: 1>,
 'sgx_version': <CheckStatus.FAILED: 0>,
 'status': <CheckStatus.FAILED: 0>,
 'ssl': <CheckStatus.PASSED: 1>,
 'versions': <CheckStatus.FAILED: 0>,
 'val_balance': <CheckStatus.PASSED: 1>,
 'internal_ports': <CheckStatus.PASSED: 1>}

## **Watchdog Client**

Client for interacting with watchdog container and its API

In [62]:
from skale_checks.adapters.watchdog import Watchdog

watchdog = Watchdog(node_ip)

In [63]:
watchdog.get_component_versions()

Response(status='ok', payload={'skale_ima_honorable-steel-rasalhague': '2.1.0', 'skale_schain_honorable-steel-rasalhague': '3.19.1', 'skale_ima_round-hasty-alsafi': '2.1.0', 'skale_schain_round-hasty-alsafi': '3.19.1', 'skale_telegraf': '1.27.4', 'skale_nginx': '1.20.2', 'skale_watchdog': '2.2.0-stable.0', 'skale_admin': '2.7.1', 'skale_redis': '6.0.10-alpine', 'skale_transaction-manager': '2.3.0', 'skale_api': '2.7.1', 'skale_bounty': '2.2.0-stable.0', 'skale_filebeat': '7.3.1', 'schain': '3.19.1', 'ima': '2.1.0', 'node-cli': '2.4.0', 'configs': '3.0.1', 'docker-lvmpy': '1.0.2-stable.0', 'os_id': 'ubuntu', 'os_version': '20.04'})

In [64]:
watchdog.get_skale_containers()

Response(status='ok', payload={'skale_ima_honorable-steel-rasalhague': {'status': 'running', 'exitCode': 0, 'finishedAt': '2024-09-10T23:14:28.704387701Z', 'version': 'skalenetwork/ima:2.1.0'}, 'skale_schain_honorable-steel-rasalhague': {'status': 'running', 'exitCode': 0, 'finishedAt': '0001-01-01T00:00:00Z', 'version': 'skalenetwork/schain:3.19.1'}, 'skale_ima_round-hasty-alsafi': {'status': 'running', 'exitCode': 0, 'finishedAt': '2024-09-10T23:31:32.726486304Z', 'version': 'skalenetwork/ima:2.1.0'}, 'skale_schain_round-hasty-alsafi': {'status': 'running', 'exitCode': 0, 'finishedAt': '0001-01-01T00:00:00Z', 'version': 'skalenetwork/schain:3.19.1'}, 'skale_telegraf': {'status': 'running', 'exitCode': 0, 'finishedAt': '2024-09-06T15:20:30.830010782Z', 'version': 'telegraf:1.27.4'}, 'skale_nginx': {'status': 'running', 'exitCode': 0, 'finishedAt': '2024-09-05T16:17:04.850577944Z', 'version': 'nginx:1.20.2'}, 'skale_watchdog': {'status': 'running', 'exitCode': 0, 'finishedAt': '0001-01

In [65]:
watchdog.get_schain_status('honorable-steel-rasalhague')

Response(status='ok', payload={'name': 'honorable-steel-rasalhague', 'healthchecks': {'config_dir': True, 'dkg': True, 'config': True, 'volume': True, 'firewall_rules': True, 'skaled_container': True, 'exit_code_ok': True, 'rpc': True, 'blocks': True, 'process': True, 'ima_container': True}})